# 타이타닉 생존자 예측: 데이터 전처리부터 모델 학습까지

이 노트북은 Kaggle의 대표적인 데이터셋인 타이타닉 생존자 예측 문제를 해결하기 위한 포괄적인 머신러닝 워크플로우를 다룹니다. 데이터 로드부터 전처리(결측치, 이상치, 범주형 데이터), 그리고 모델 학습 및 평가에 이르는 전 과정을 단계별로 상세히 설명합니다.

### 1. 라이브러리 임포트

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import seaborn as sns # 시각화를 위해 추가

from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier # 중요도 파악을 위해 원본 스크립트에 있었음
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestClassifier # 최종 모델

import os # 파일 경로 지정을 위해

### 2. 데이터 로드 및 초기 탐색

`train.csv`와 `test.csv` 파일을 불러와 데이터의 기본적인 구조와 내용을 확인합니다.

In [ ]:
try:
    train_df = pd.read_csv("./data/타이타닉2/train.csv")
    test_df = pd.read_csv("./data/타이타닉2/test.csv")
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("오류: './data/타이타닉2/' 폴더 또는 파일이 없습니다.")
    print("노트북이 있는 폴더(02_practice/0716) 아래에 'data/타이타닉2/' 폴더를 만들고 train.csv와 test.csv를 넣어주세요.")
    train_df, test_df = None, None

if train_df is not None:
    print("
훈련 데이터 shape:", train_df.shape)
    print("테스트 데이터 shape:", test_df.shape)
    print("
훈련 데이터 상위 5행:")
    print(train_df.head())
    print("
훈련 데이터 정보:")
    train_df.info()
    print("
훈련 데이터 통계 요약:")
    print(train_df.describe())

### 3. 불필요한 열 제거

`PassengerId`, `Name`, `SibSp`, `Parch`는 예측에 직접적인 도움이 되지 않거나 다른 특성으로 대체될 수 있습니다. `Cabin`은 결측치가 너무 많아 제거하는 것이 합리적입니다.

In [ ]:
if train_df is not None:
    # 원본 데이터프레임을 직접 수정하지 않기 위해 복사본을 만듭니다.
    train_df_processed = train_df.copy()
    test_df_processed = test_df.copy()

    drop_columns = ['PassengerId', 'Name', 'SibSp', 'Parch', 'Cabin']
    train_df_processed = train_df_processed.drop(columns=drop_columns)
    test_df_processed = test_df_processed.drop(columns=drop_columns)

    print("
열 제거 후 훈련 데이터 상위 5행:")
    print(train_df_processed.head())
    print("열 제거 후 훈련 데이터 shape:", train_df_processed.shape)

### 4. 결측치 처리

`Age`와 `Embarked` 열에 결측치가 존재합니다. `Age`는 평균값으로 대체하고, `Embarked`는 결측치가 적으므로 해당 행을 제거합니다.

In [ ]:
if train_df_processed is not None:
    print("
결측치 확인 (처리 전):")
    print(train_df_processed.isna().sum())

    # Age 결측치 평균으로 대체
    age_mean = train_df_processed["Age"].mean() 
    train_df_processed['Age'].fillna(age_mean, inplace=True) 
    test_df_processed['Age'].fillna(age_mean, inplace=True)

    # Embarked 결측치 행 제거
    train_df_processed = train_df_processed.dropna(axis=0, how='any') 
    # 테스트 데이터셋의 Embarked 결측치도 처리 (원본 스크립트와 동일하게 행 제거)
    test_df_processed = test_df_processed.dropna(axis=0, how='any')

    print("
결측치 확인 (처리 후):")
    print(train_df_processed.isna().sum())
    print(test_df_processed.isna().sum())

### 5. 이상치 처리 (IQR 방식)

`Fare`와 `Age` 열에 이상치가 있을 수 있습니다. IQR(사분위수 범위) 방식을 사용하여 이상치를 탐지하고, 해당 값을 상한/하한 경계값으로 대체(capping)합니다.

In [ ]:
def outfliers_iqr(data_series):
    q1, q3 = np.percentile(data_series, [25, 75])  
    iqr = q3 - q1 
    lower_bound = q1 - iqr * 1.5
    upper_bound = q3 + iqr * 1.5
    return lower_bound, upper_bound

if train_df_processed is not None:
    for col in ['Fare', 'Age']:
        lower, upper = outfliers_iqr(train_df_processed[col])
        train_df_processed[col][train_df_processed[col] < lower] = lower 
        train_df_processed[col][train_df_processed[col] > upper] = upper 

        lower_test, upper_test = outfliers_iqr(test_df_processed[col])
        test_df_processed[col][test_df_processed[col] < lower_test] = lower_test 
        test_df_processed[col][test_df_processed[col] > upper_test] = upper_test 

    # 이상치 처리 후 boxplot (주석 해제하여 확인 가능)
    # plt.figure(figsize=(10, 5))
    # plt.subplot(1, 2, 1)
    # train_df_processed['Fare'].plot(kind='box', title='Fare after Outlier Treatment')
    # plt.subplot(1, 2, 2)
    # train_df_processed['Age'].plot(kind='box', title='Age after Outlier Treatment')
    # plt.tight_layout()
    # plt.show()

### 6. 범주형 데이터 원-핫 인코딩

`Sex`와 `Embarked`는 범주형 데이터이므로, 머신러닝 모델이 이해할 수 있도록 원-핫 인코딩을 수행합니다.

In [ ]:
if train_df_processed is not None:
    train_df_encoded = pd.get_dummies(train_df_processed)
    test_df_encoded = pd.get_dummies(test_df_processed)

    print("
원-핫 인코딩 후 훈련 데이터 상위 5행:")
    print(train_df_encoded.head())
    print("원-핫 인코딩 후 훈련 데이터 컬럼:")
    print(train_df_encoded.columns)

### 7. 특성(X)과 타겟(y) 분리

훈련 데이터에서 예측 대상인 `Survived` 열을 타겟(`y`)으로, 나머지 열을 특성(`X`)으로 분리합니다.

In [ ]:
if train_df_encoded is not None:
    X_train = train_df_encoded.drop(columns=['Survived'])
    y_train = train_df_encoded['Survived']

    # 테스트 데이터셋은 Survived 컬럼이 없으므로 그대로 사용
    X_test = test_df_encoded

    print("
훈련 특성 데이터 shape:", X_train.shape)
    print("훈련 타겟 데이터 shape:", y_train.shape)
    print("테스트 특성 데이터 shape:", X_test.shape)

### 8. 모델 학습 및 평가 (RandomForestClassifier)

전처리된 데이터를 사용하여 `RandomForestClassifier` 모델을 학습시키고, 훈련 데이터에 대한 정확도를 평가합니다.

In [ ]:
if X_train is not None:
    model = RandomForestClassifier(n_estimators=100, random_state=0) # 재현성을 위해 random_state 추가
    model.fit(X_train, y_train)

    train_score = model.score(X_train, y_train)
    print(f"훈련 세트 정확도: {train_score:.4f}")

    # 테스트 데이터에 대한 예측 (실제 Kaggle 제출 시 사용)
    # test_pred = model.predict(X_test)
    # print("테스트 데이터 예측 (일부):", test_pred[:10])

### 9. (선택 사항) 상관관계 분석

특성 간의 상관관계를 히트맵으로 시각화하여 데이터의 구조를 파악할 수 있습니다. 특성의 개수가 많을 경우 `pairplot`은 비효율적이므로, 상관관계 히트맵이 더 적합합니다.

In [ ]:
if train_df_encoded is not None:
    correlation_matrix = train_df_encoded.corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, 
                annot=True, 
                cmap='coolwarm', 
                fmt='.2f', 
                linewidths=.5 
               )
    plt.xticks(rotation=45, ha='right') 
    plt.yticks(rotation=0)
    plt.title('타이타닉 데이터셋 특성 간 상관관계 히트맵')
    plt.tight_layout() 
    plt.show()